## Sustainable and Entrepreneurial Finance- Homework #1

In [2]:
#Necessary libraries to run the code
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


### Question 3- Optimizing the portfolio

In [8]:
from scipy.optimize import minimize

file_path = r"data_h1\filtered_data\DS_RI_USD_M.xlsx"

#Filtering the data frame and using data only from 2014 to 2024
df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)
df_returns = df_returns.select_dtypes(include=[np.number])
df_returns.columns = pd.to_datetime(df_returns.columns, format='%Y-%m-%d', errors='coerce')
df_returns=df_returns.loc[:, "2014-01-31":"2024-12-31"]

#Computing the mean and the covariance matrix 
mu_hat = df_returns.mean(axis=1,skipna=True)
Sigma_hat = df_returns.T.cov()

# Number of assets
n_assets = len(mu_hat)

# Convert expected returns and covariance matrix to NumPy arrays
mu_array = mu_hat.values.reshape(-1, 1)  # Column vector
Sigma_array = Sigma_hat.values  # Covariance matrix

# Constraint: sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bounds: No short selling (weights >= 0)
bounds = [(0, 1) for _ in range(n_assets)]

### Step 1: Compute Minimum Variance Portfolio (MVP)
def portfolio_variance(w, Sigma):
    return w.T @ Sigma @ w

# Initial guess (equal weights)
w0 = np.ones(n_assets) / n_assets

# Solve for minimum variance portfolio
# Remove NaN values from the covariance matrix
#Sigma_array = np.nan_to_num(Sigma_array)  #changed 

mvp_result = minimize(portfolio_variance, w0, args=(Sigma_array,), method='SLSQP', bounds=bounds, constraints=constraints)
print(mvp_result)
w_mvp = mvp_result.x  # Optimal weights

# Compute return and variance of the MVP
mu_mvp = w_mvp.T @ mu_array
sigma_mvp = np.sqrt(mvp_result.fun)

# Compute Maximum Return Portfolio
def negative_return(w, mu):
    return -w.T @ mu

max_return_result = minimize(negative_return, w0, args=(mu_array,), method='SLSQP', bounds=bounds, constraints=constraints)
w_max_ret = max_return_result.x  # Optimal weights

# Compute return and variance of the max return portfolio
mu_max_ret = w_max_ret.T @ mu_array
sigma_max_ret = np.sqrt(w_max_ret.T @ Sigma_array @ w_max_ret)

## Compute Efficient Frontier (10 to 20 portfolios)
# Define target returns between mu_mvp and mu_max_ret (increments of 0.5%)
target_returns = np.linspace(mu_mvp, mu_max_ret, num=15)  

efficient_frontier = []

for target_return in target_returns:
    # Constraint: Portfolio return = target return
    return_constraint = {'type': 'eq', 'fun': lambda w: w.T @ mu_array - target_return}
    
    # Solve optimization problem
    result = minimize(portfolio_variance, w0, args=(Sigma_array,), method='SLSQP', bounds=bounds, constraints=[constraints, return_constraint])
    
    # Store results
    w_optimal = result.x
    sigma_optimal = np.sqrt(result.fun)
    efficient_frontier.append((target_return, sigma_optimal, w_optimal))

# Convert results to a DataFrame for visualization
efficient_frontier_df = pd.DataFrame(efficient_frontier, columns=['Target Return', 'Risk (Std Dev)', 'Weights'])

# Display results
display(efficient_frontier_df)

# Save the efficient frontier DataFrame to an Excel file
efficient_frontier_df.to_excel("efficient_frontier.xlsx", index=False)

C:\Users\ameli\AppData\Local\Temp\ipykernel_10072\1922365997.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)


 message: Inequality constraints incompatible
 success: False
  status: 4
     fun: nan
       x: [ 1.608e-03  1.608e-03 ...  1.608e-03  1.608e-03]
     nit: 1
     jac: [       nan        nan ...        nan        nan]
    nfev: 623
    njev: 1


,Target Return,Risk (Std Dev),Weights
0,[0.006075276155823957],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
1,[0.007927039702246673],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
2,[0.009778803248669386],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
3,[0.011630566795092102],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
4,[0.013482330341514817],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
5,[0.015334093887937533],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
6,[0.017185857434360248],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
7,[0.01903762098078296],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
8,[0.02088938452720568],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."
9,[0.02274114807362839],NaN,"[0.001607717041800643, 0.001607717041800643, 0..."


PermissionError: [Errno 13] Permission denied: 'efficient_frontier.xlsx'

In [7]:
# Optimized computation of the covariance matrix using the provided formula
from scipy.optimize import minimize

file_path = r"data_h1\filtered_data\DS_RI_USD_M.xlsx"

#Filtering the data frame and using data only from 2014 to 2024
df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)
df_returns = df_returns.select_dtypes(include=[np.number])
df_returns.columns = pd.to_datetime(df_returns.columns, format='%Y-%m-%d', errors='coerce')
df_returns=df_returns.loc[:, "2014-01-31":"2024-12-31"]
returns_data = df_returns.T  # Transpose so rows correspond to time, columns to companies

# Compute mean for each company, considering only available data
means = returns_data.mean(skipna=True)

# Initialize covariance matrix
n_companies = returns_data.shape[1]
cov_matrix_custom = np.zeros((n_companies, n_companies))
# Initialize covariance matrix with NaNs
cov_matrix_optimized = np.full((n_companies, n_companies), np.nan)

# Convert to NumPy array for faster operations
returns_array = returns_data.to_numpy()

# Iterate over unique pairs of companies
for i in range(n_companies):
    for j in range(i, n_companies):  # Compute only upper triangle (symmetry)
        # Get valid indices where both series have data
        valid_mask = ~np.isnan(returns_array[:, i]) & ~np.isnan(returns_array[:, j])
        common_returns = returns_array[valid_mask, :]

        if common_returns.shape[0] > 2:  # Ensure sufficient data points
            Ri = common_returns[:, i]
            Rj = common_returns[:, j]
            
            # Compute means over the common period
            Ri_mean = np.mean(Ri)
            Rj_mean = np.mean(Rj)

            # Compute covariance using the given formula
            covariance = np.sum((Ri - Ri_mean) * (Rj - Rj_mean)) / (len(Ri) - 2)

            # Store results
            cov_matrix_optimized[i, j] = covariance
            cov_matrix_optimized[j, i] = covariance  # Symmetric matrix

# Convert to DataFrame
cov_matrix_optimized_df = pd.DataFrame(cov_matrix_optimized, index=returns_data.columns, columns=returns_data.columns)

# Display the optimized covariance matrix
display(cov_matrix_optimized_df)

print(cov_matrix_optimized_df.isna().sum().sum())


C:\Users\ameli\AppData\Local\Temp\ipykernel_26940\145342653.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)


NAME,STRABAG SE,FLUGHAFEN WIEN,RAIFFEISEN BANK INTL.,ERSTE GROUP BANK,TELEKOM AUSTRIA,ANDRITZ,OMV,VERBUND,WIENERBERGER,VIENNA INSURANCE GROUP A,...,NIBE INDUSTRIER,ELECTROLUX B,ATLAS COPCO A,MODERN TIMES GROUP MTG B,WIHLBORGS FASTIGHETER,BOLIDEN ORD SHS,AUTOLIV,PARTNER COMMS.ADR 1:1 DEAD - DELIST.22/09/23,TEVA PHARMACEUTICAL INDUSTRIES ADR 1:1,VEON ADR 1:25
NAME,,,,,,,,,,,,,,,,,,,,,
STRABAG SE,0.004947,0.002362,0.003989,0.003734,0.001430,0.002782,0.003798,0.002937,0.003236,0.002015,...,0.003021,0.002570,0.001970,0.003197,0.002321,0.002760,0.003346,0.001846,0.002631,0.002551
FLUGHAFEN WIEN,0.002362,0.005846,0.003160,0.003358,0.001327,0.002471,0.003491,0.002559,0.002156,0.002275,...,0.002297,0.002051,0.001263,0.002354,0.001132,0.001895,0.002521,0.001544,0.000450,0.001971
RAIFFEISEN BANK INTL.,0.003989,0.003160,0.013040,0.008189,0.003126,0.004910,0.006724,0.003002,0.005882,0.004150,...,0.003924,0.004606,0.004300,0.004226,0.003717,0.004211,0.006093,0.005807,0.002444,0.005805
ERSTE GROUP BANK,0.003734,0.003358,0.008189,0.010284,0.002194,0.003832,0.006426,0.003450,0.005487,0.004279,...,0.003836,0.004022,0.002914,0.003528,0.003873,0.003746,0.005076,0.004151,0.003860,0.006254
TELEKOM AUSTRIA,0.001430,0.001327,0.003126,0.002194,0.003400,0.001986,0.002172,0.001944,0.002060,0.001786,...,0.001520,0.001599,0.001486,0.001617,0.002186,0.001723,0.001259,0.000904,0.000379,0.002302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BOLIDEN ORD SHS,0.002760,0.001895,0.004211,0.003746,0.001723,0.003624,0.003528,0.002776,0.004165,0.002626,...,0.004736,0.004087,0.004194,0.004045,0.003475,0.009414,0.004087,0.001697,0.001879,0.003196
AUTOLIV,0.003346,0.002521,0.006093,0.005076,0.001259,0.004129,0.004382,0.002781,0.005904,0.003300,...,0.004506,0.004501,0.003741,0.003744,0.003055,0.004087,0.008968,0.001905,0.003848,0.003570
PARTNER COMMS.ADR 1:1 DEAD - DELIST.22/09/23,0.001846,0.001544,0.005807,0.004151,0.000904,0.002377,0.003526,0.002655,0.002854,0.002745,...,0.001758,0.001547,0.001320,0.002954,0.002116,0.001697,0.001905,0.014837,0.002398,0.002577


130


In [ ]:
from scipy.optimize import minimize

file_path = r"data_h1\filtered_data\DS_RI_USD_M.xlsx"

#Filtering the data frame and using data only from 2014 to 2024
df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)
df_returns = df_returns.select_dtypes(include=[np.number])
df_returns.columns = pd.to_datetime(df_returns.columns, format='%Y-%m-%d', errors='coerce')
df_returns=df_returns.loc[:, "2014-01-31":"2024-12-31"]

#Computing the mean and the covariance matrix 
mu_hat = df_returns.mean(axis=1,skipna=True)
#Sigma_hat = df_returns.T.cov()

Sigma_hat=cov_matrix_optimized_df
# Number of assets
n_assets = len(mu_hat)

# Convert expected returns and covariance matrix to NumPy arrays
mu_array = mu_hat.values.reshape(-1, 1)  # Column vector
Sigma_array = Sigma_hat.values  # Covariance matrix

# Constraint: sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bounds: No short selling (weights >= 0)
bounds = [(0, 1) for _ in range(n_assets)]

### Step 1: Compute Minimum Variance Portfolio (MVP)
def portfolio_variance(w, Sigma):
    return w.T @ Sigma @ w

# Initial guess (equal weights)
w0 = np.ones(n_assets) / n_assets

# Solve for minimum variance portfolio
# Remove NaN values from the covariance matrix
Sigma_array = np.nan_to_num(Sigma_array)  #remove Nan for non-overlapping returns for some companies

mvp_result = minimize(portfolio_variance, w0, args=(Sigma_array,), method='SLSQP', bounds=bounds, constraints=constraints)
print(mvp_result)
w_mvp = mvp_result.x  # Optimal weights

# Compute return and variance of the MVP
mu_mvp = w_mvp.T @ mu_array
sigma_mvp = np.sqrt(mvp_result.fun)

# Compute Maximum Return Portfolio
def negative_return(w, mu):
    return -w.T @ mu

max_return_result = minimize(negative_return, w0, args=(mu_array,), method='SLSQP', bounds=bounds, constraints=constraints)
w_max_ret = max_return_result.x  # Optimal weights

# Compute return and variance of the max return portfolio
mu_max_ret = w_max_ret.T @ mu_array
sigma_max_ret = np.sqrt(w_max_ret.T @ Sigma_array @ w_max_ret)

## Compute Efficient Frontier (10 to 20 portfolios)
# Define target returns between mu_mvp and mu_max_ret (increments of 0.5%)
target_returns = np.linspace(mu_mvp, mu_max_ret, num=15)  

efficient_frontier = []

for target_return in target_returns:
    # Constraint: Portfolio return = target return
    return_constraint = {'type': 'eq', 'fun': lambda w: w.T @ mu_array - target_return}
    
    # Solve optimization problem
    result = minimize(portfolio_variance, w0, args=(Sigma_array,), method='SLSQP', bounds=bounds, constraints=[constraints, return_constraint])
    
    # Store results
    w_optimal = result.x
    sigma_optimal = np.sqrt(result.fun)
    efficient_frontier.append((target_return, sigma_optimal, w_optimal))

# Convert results to a DataFrame for visualization
efficient_frontier_df = pd.DataFrame(efficient_frontier, columns=['Target Return', 'Risk (Std Dev)', 'Weights'])

# Display results
display(efficient_frontier_df)

# Save the efficient frontier DataFrame to an Excel file
efficient_frontier_df.to_excel("data_h1/results/efficient_frontier.xlsx", index=False)

C:\Users\ameli\AppData\Local\Temp\ipykernel_26940\738553426.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0005885598988495884
       x: [ 7.839e-18  0.000e+00 ...  0.000e+00  2.595e-18]
     nit: 36
     jac: [ 1.557e-03  1.652e-03 ...  1.421e-03  2.950e-03]
    nfev: 22429
    njev: 36


,Target Return,Risk (Std Dev),Weights
0,[0.002765904591869544],0.024190,"[0.0, 2.335934930105148e-18, 0.0, 2.7855050630..."
1,[0.00485405182143186],0.024362,"[0.0, 6.04704007503429e-18, 0.0, 4.17250862063..."
2,[0.0069421990509941755],0.025196,"[0.0, 0.0, 0.0, 4.737991987342406e-18, 0.0, 0...."
3,[0.009030346280556493],0.026775,"[0.0, 1.6385178864841692e-18, 0.0, 0.0, 1.3091..."
4,[0.011118493510118808],0.028292,"[7.479958893148199e-19, 0.0, 0.0, 0.0, 0.0, 0...."
5,[0.013206640739681124],0.030702,"[1.3384225718692059e-17, 0.0, 0.0, 0.0, 0.0, 5..."
6,[0.015294787969243441],0.033696,"[8.604576067675425e-18, 9.51052323186589e-19, ..."
7,[0.017382935198805755],0.037655,"[8.412075089758116e-18, 0.0, 1.717147244061052..."
8,[0.01947108242836807],0.043287,"[0.0, 3.3412129364732183e-18, 0.0, 7.428874778..."
9,[0.021559229657930386],0.050398,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.468..."
